In [1]:
import pandas as pd
import numpy as np

In [2]:
val_pct = 0.2
batch_size = 100
hidden_layer_size = 12
learning_rate = 0.001

In [3]:
data=pd.read_csv('train.csv',parse_dates=['date_time'])

In [4]:
#data.head()

In [5]:
#data.info()

In [6]:
data['date_time']

0      2010-03-10 18:00:00
1      2010-03-10 19:00:00
2      2010-03-10 20:00:00
3      2010-03-10 21:00:00
4      2010-03-10 22:00:00
               ...        
7106   2010-12-31 20:00:00
7107   2010-12-31 21:00:00
7108   2010-12-31 22:00:00
7109   2010-12-31 23:00:00
7110   2011-01-01 00:00:00
Name: date_time, Length: 7111, dtype: datetime64[ns]

In [7]:
data['date_time'].iloc[0].hour

18

In [8]:
data['month']=data['date_time'].apply(lambda x: x.month)
data['hour']=data['date_time'].apply(lambda x: x.hour)

In [9]:
data

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,month,hour
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7,3,18
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9,3,19
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1,3,20
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2,3,21
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8,3,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,2010-12-31 20:00:00,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,1.3,5.1,191.1,12,20
7107,2010-12-31 21:00:00,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,1.4,5.8,221.3,12,21
7108,2010-12-31 22:00:00,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,1.6,5.2,227.4,12,22
7109,2010-12-31 23:00:00,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,1.5,4.6,199.8,12,23


In [10]:
def get_season(row):
    if int(row) == 12 or int(row) <= 2:
        return 'Winter'
    elif int(row) > 2 and int(row) <= 5:
        return 'Spring'
    elif int(row) > 2 and int(row) <= 5:
        return 'Summer'
    else:
        return "Fall"
    
def get_timeOfDay(row):
    if int(row) > 4 and int(row) <= 10:
        return "Morning"
    elif int(row) > 11 and int(row) <= 20:
        return "Daytime"
    else:
        return "Nighttime"

    
    
data['season']=data['month'].apply(get_season)
data['time_of_day']=data['hour'].apply(get_timeOfDay)

In [11]:
data

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,month,hour,season,time_of_day
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7,3,18,Spring,Daytime
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9,3,19,Spring,Daytime
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1,3,20,Spring,Daytime
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2,3,21,Spring,Nighttime
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8,3,22,Spring,Nighttime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,2010-12-31 20:00:00,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,1.3,5.1,191.1,12,20,Winter,Daytime
7107,2010-12-31 21:00:00,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,1.4,5.8,221.3,12,21,Winter,Nighttime
7108,2010-12-31 22:00:00,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,1.6,5.2,227.4,12,22,Winter,Nighttime
7109,2010-12-31 23:00:00,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,1.5,4.6,199.8,12,23,Winter,Nighttime


In [12]:
#data.info()

In [13]:
#data.shape

In [14]:
#"High"

In [15]:
#data.isnull().sum()

In [16]:
## Metric
# MEan Squared Error --> 0

In [17]:
#data.corr()

In [18]:
carbon_monoxide_col = data.pop('target_carbon_monoxide')
benzene_col = data.pop('target_benzene')
nitrogen_oxide_col = data.pop('target_nitrogen_oxides')


data

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,month,hour,season,time_of_day
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,3,18,Spring,Daytime
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,3,19,Spring,Daytime
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,3,20,Spring,Daytime
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,3,21,Spring,Nighttime
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,3,22,Spring,Nighttime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,2010-12-31 20:00:00,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,12,20,Winter,Daytime
7107,2010-12-31 21:00:00,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,12,21,Winter,Nighttime
7108,2010-12-31 22:00:00,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,12,22,Winter,Nighttime
7109,2010-12-31 23:00:00,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,12,23,Winter,Nighttime


In [19]:
data = pd.get_dummies(data, drop_first = True)

data

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,month,hour,season_Spring,season_Winter,time_of_day_Morning,time_of_day_Nighttime
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,3,18,1,0,0,0
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,3,19,1,0,0,0
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,3,20,1,0,0,0
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,3,21,1,0,0,1
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,3,22,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,2010-12-31 20:00:00,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,12,20,0,1,0,0
7107,2010-12-31 21:00:00,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,12,21,0,1,0,1
7108,2010-12-31 22:00:00,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,12,22,0,1,0,1
7109,2010-12-31 23:00:00,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,12,23,0,1,0,1


In [20]:
drop_columns = ['date_time', 'month', 'hour']
data.drop(drop_columns, axis=1)

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,season_Spring,season_Winter,time_of_day_Morning,time_of_day_Nighttime
0,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,1,0,0,0
1,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,1,0,0,0
2,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,1,0,0,0
3,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,1,0,0,1
4,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7106,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,0,1,0,0
7107,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,0,1,0,1
7108,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,0,1,0,1
7109,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,0,1,0,1


In [21]:
labels = pd.DataFrame([carbon_monoxide_col, benzene_col, nitrogen_oxide_col]).transpose()

labels = labels.transpose()
labels


,0,1,2,3,4,5,6,7,8,9,...,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110
target_carbon_monoxide,2.5,2.1,2.2,2.2,1.5,1.2,1.2,1.0,0.9,0.6,...,0.9,1.1,1.4,1.5,1.2,1.3,1.4,1.6,1.5,1.4
target_benzene,12.0,9.9,9.2,9.7,6.4,4.4,3.7,3.4,2.2,1.6,...,4.3,5.8,7.4,5.9,4.9,5.1,5.8,5.2,4.6,4.1
target_nitrogen_oxides,167.7,98.9,127.1,177.2,121.8,88.1,59.5,63.9,46.4,43.0,...,165.9,199.4,228.5,206.1,211.0,191.1,221.3,227.4,199.8,186.5


In [22]:
# Create validation split by taking a percentage of the training set:

def split_indices(dataset, pct):
    length = len(dataset)
    split = int(length*pct)
    indices = list(range(length))
    np.random.shuffle(indices)
    #indices = np.random.permutation(n)
    return indices[split:], indices[:split]

train_indices, val_indices = split_indices(data, val_pct)

In [23]:
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# Use the SubsetRandomSampler to randomly sample the training dataset for 
# training and validation data. We will feed this into the dataloader below.

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Create dataloader objects - will be used during training and inference
# to iterate over the data.

train_loader = torch.utils.data.DataLoader(data, batch_size = batch_size, \
                                           sampler = train_sampler)
val_loader = torch.utils.data.DataLoader(data, batch_size = batch_size, \
                                         sampler = val_sampler)

In [24]:
#dataiter = iter(val_loader)
#features, labels = dataiter.next()

In [26]:
import torch.nn as nn

# Here we are going to create our neural network class by inheiriting the 
# nn.Module class. The main functions to notice are the initialization function 
# __init__() and forward propagation function forward().

class Model(nn.Module):

  # We initialize our neural net by defining the input size, number of nodes
  # in hidden layers, and the total number of classes we are detecting.
  def __init__(self, input_size, hidden_size, num_classes):

    # IMPORTANT: This is often forgotten, but make sure to make a super call.
    super(Model, self).__init__()

    # 1st Fully-Connected Layer: 80 (input nodes or features) -> 8 (hidden nodes)
    self.fc1 = nn.Linear(input_size, hidden_size)
    
    # Applying activation function
    self.relu = nn.ReLU()

    # 2nd Fully-Connected Layer: 16 (hidden nodes) -> 16 (hidden nodes)
    self.fc2 = nn.Linear(hidden_size, hidden_size)

    # Output Layer: 16 (hidden nodes) -> 1 (output class)
    self.fc3 = nn.Linear(hidden_size, num_classes)
  
  # Below we construct the output of the forward propagation pass of the neural
  # net using the layers and activation functions defined in the Net constructor.
  def forward(self, x):
        
    if np.isnan(np.sum(x.data.numpy())):
        print("nan0")
        
    

    # Feeding input layer into the hidden layer and activation function
    out = self.fc1(x)
    
    if np.isnan(np.sum(out.data.numpy())):
        print("nan1")

    out = self.relu(out)
    
    
    # Feeding output from first hidden layer to second layer
    out = self.fc2(out)
    
    
    out = self.relu(out)
    

    # Feeding output from second hidden layer to output layer
    out = self.fc3(out)
    

    # Returning output from the (10) output layer neurons
    return out

In [27]:
# Create a neural network object with the specified number of input neurons,
# hidden neurons, and output neurons (or total classes)

model = Model(12, hidden_layer_size, 3)

# Define Loss Function and Optimizer

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


#torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma=0.1, last_epoch=-1)

#Decays the learning rate of each parameter group by gamma every step_size epochs. Notice 
#that such decay can happen simultaneously with other changes to the learning rate from outside 
#this scheduler. When last_epoch=-1, sets initial lr as lr.

#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.004, last_epoch = -0.01)

In [29]:
list(model.parameters())

[Parameter containing:
 tensor([[ 2.2375e-01, -2.5595e-01,  1.5194e-01, -2.4889e-01, -1.0839e-01,
           1.4674e-02,  1.1207e-01,  2.6663e-01,  1.6312e-01, -2.3802e-01,
           2.8662e-01,  3.2024e-02],
         [-1.1939e-01, -8.6058e-02,  1.6927e-01, -9.1414e-02,  2.3388e-01,
          -1.2794e-01,  6.0121e-02, -1.7894e-01,  1.6321e-01,  5.4590e-02,
           1.9634e-01, -2.7721e-01],
         [-2.2185e-01,  7.8488e-02,  1.4300e-01,  2.4590e-01, -2.3046e-01,
          -2.7562e-01, -2.3854e-01, -1.0962e-01, -9.0419e-02, -2.7961e-01,
          -1.9077e-01,  1.2230e-01],
         [-9.1337e-03, -2.7850e-01, -2.2545e-01, -1.5673e-01,  4.8896e-02,
           3.6379e-02, -1.0082e-01,  2.1639e-01,  2.1425e-01,  1.6003e-01,
          -1.3265e-01, -8.1371e-02],
         [-2.0810e-01,  3.9003e-02,  8.0346e-02,  2.2614e-01,  3.1660e-02,
          -1.9956e-01, -1.6683e-01,  2.7788e-01,  1.5866e-01, -9.5614e-02,
          -2.0370e-01,  4.5001e-02],
         [-1.3066e-02,  2.6902e-01,  1.393

In [ ]:
history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        
        # Training Phase 
        for batch in train_loader:
            features, labels = batch 
            output = model(images)                  # Generate predictions
            loss = criterion(outputs, labels) # Calculate loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Validation phase
        for batch in val_loader:
            features, labels = batch 
            output = model(images)                  # Generate predictions
            loss = criterion(outputs, labels)
            batch_losses = [x['val_loss'] for x in outputs]
            epoch_loss = torch.stack(batch_losses).mean()
            print({'val_loss': epoch_loss.item()})
            
        
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)